In [1]:
from keras.models import Model
from keras.layers import Dense,Flatten
from keras.applications import vgg16
from keras import backend as K
from keras.preprocessing import image
from keras.models import Sequential
from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
import tensorflow as tf
import os
import tensorboard

Using TensorFlow backend.


In [2]:
input_tensor = Input(shape=(224,224,3))
model = vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

In [3]:
model

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [5]:
model_ft = Sequential()

for layer in model.layers:
    model_ft.add(layer)


In [6]:
model_ft.add(Flatten())
model_ft.add(Dense(512, activation='relu'))
model_ft.add(Dropout(0.5))
model_ft.add(Dense(512, activation='relu'))
model_ft.add(Dropout(0.5))
model_ft.add(Dense(8, activation='softmax'))

In [7]:
model_ft.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [8]:
import keras

sgd_apna =keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0001, amsgrad=False)
model_ft.compile(optimizer=sgd_apna, loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
# import pickle

In [10]:
# # ValData= []
# with open("ValData.pkl","rb") as f:
#     ValData = pickle.load(f)

In [11]:
# TrainData= []
# with open("TrainData.pkl","rb") as f:
#     TrainData = pickle.load(f)

In [12]:
# len(TrainData)

In [13]:
# imgData=[]
# imgLabels=[]
# validityData = []
# validityLabels = []
# for i in range(len(TrainData)):
#     imgData.append(TrainData[i][0])
#     imgLabddels.append(TrainData[i][1])
    
# for i in range(len(ValData)):
#     validityData.append(ValData[i][0])
#     validityLabels.append(ValData[i][1])


In [14]:
# def getImages(img_path):
#     img = image.load_img(img_path, target_size=(224, 224))
#     x = image.img_to_array(img)
#     return x

In [15]:
# finalTrainData= []
# count = 0
# for i in range(len(imgData)):
#     count += 1
#     print(count)
#     #print(imgData[i])
#     finalTrainData.append(getImages(imgData[i]))

In [16]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
batch_size = 32

#train_datagen = ImageDataGenerator()
train_datagen = ImageDataGenerator(
        #rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        #horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        'train',  # this is the target directory
        target_size=(224, 224),  
        batch_size=batch_size,
        class_mode='categorical'
) 



Found 17378 images belonging to 8 classes.


In [17]:
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [18]:
validation_generator = val_datagen.flow_from_directory(
        'test',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
checkpointer = ModelCheckpoint(filepath='weights_latest_8.hdf5', verbose=1, save_best_only=True)
red=ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, mode='auto', min_delta=0.001, cooldown=0, min_lr=0)


Found 6488 images belonging to 8 classes.


In [19]:
callbacks = [keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)]

history = model_ft.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples//train_generator.batch_size ,
      epochs=400,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,callbacks=[red,checkpointer]
      )
 
# Save the model
model_ft.save('final_model.h5')

Epoch 1/400
543/543 [==============================] - 492s 906ms/step - loss: 2.3294 - acc: 0.2173 - val_loss: 2.1963 - val_acc: 0.1974

Epoch 00001: val_loss improved from inf to 2.19628, saving model to weights_latest_8.hdf5
Epoch 2/400
543/543 [==============================] - 524s 966ms/step - loss: 1.3785 - acc: 0.4749 - val_loss: 1.9542 - val_acc: 0.2833

Epoch 00002: val_loss improved from 2.19628 to 1.95417, saving model to weights_latest_8.hdf5
Epoch 3/400
543/543 [==============================] - 625s 1s/step - loss: 1.0032 - acc: 0.5855 - val_loss: 1.6495 - val_acc: 0.4212

Epoch 00003: val_loss improved from 1.95417 to 1.64953, saving model to weights_latest_8.hdf5
Epoch 4/400
543/543 [==============================] - 855s 2s/step - loss: 0.7106 - acc: 0.7062 - val_loss: 1.3113 - val_acc: 0.6401

Epoch 00004: val_loss improved from 1.64953 to 1.31134, saving model to weights_latest_8.hdf5
Epoch 5/400
543/543 [==============================] - 1196s 2s/step - loss: 0.410

KeyboardInterrupt: 